In [1]:
import pandas as pd

In [2]:
#********************************************************************************************************

In [4]:
data = pd.read_csv('data.csv')
data.head(2)

,imdb_id,popularity,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year
0,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,5562,6.5,2015
1,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,6185,7.1,2015


In [17]:
data.columns

Index(['imdb_id', 'popularity', 'budget', 'revenue', 'original_title', 'cast',
       'director', 'tagline', 'overview', 'runtime', 'genres',
       'production_companies', 'release_date', 'vote_count', 'vote_average',
       'release_year'],
      dtype='object')

**1. У какого фильма из списка самый большой бюджет?**

In [14]:
data.loc[data.budget.idxmax()].original_title

"The Warrior's Way"

**2. Какой из фильмов самый длительный (в минутах)**

In [20]:
data.loc[data.runtime.idxmax()].original_title

'Gods and Generals'

**3. Какой из фильмов самый короткий (в минутах)**

In [23]:
data.loc[data.runtime.idxmin()].original_title

'Winnie the Pooh'

**4. Средняя длительность фильма?**

In [28]:
round(data.runtime.mean())

110

**5. Средняя длительность фильма по медиане?**

In [29]:
round(data.runtime.median())

106

**6. Какой самый прибыльный фильм?**

In [39]:
data.loc[(data.revenue - data.budget).idxmax()].original_title

'Avatar'

**7. Какой фильм самый убыточный?**

In [40]:
data.loc[(data.revenue - data.budget).idxmin()].original_title

"The Warrior's Way"

**8. Сколько всего фильмов в прибыли?**

In [44]:
len(data.loc[(data.revenue - data.budget)> 0])

1478

In [48]:
((data.revenue - data.budget)>0).sum()

1478

**9. Самый прибыльный фильм в 2008 году?**

In [60]:
year = pd.to_datetime(data.release_date).dt.year
data2008 = data[year == 2008]
data2008.loc[(data2008.revenue - data2008.budget).idxmax()].original_title

'The Dark Knight'

**10. Самый убыточный фильм за период с 2012 по 2014 (включительно)?**

In [110]:
year.isin([2012,2013,2014])

0       False
1       False
2       False
3       False
4       False
        ...  
1885    False
1886    False
1887    False
1888    False
1889    False
Name: release_date, Length: 1890, dtype: bool

In [111]:
year = pd.to_datetime(data.release_date).dt.year
data2012_2014 = data[year.isin([2012,2013,2014])]
data2012_2014.loc[(data2012_2014.revenue - data2012_2014.budget).idxmin()].original_title

'The Lone Ranger'

**11. Какого жанра фильмов больше всего?**

In [104]:
total_genres_list = []
for g in data.genres: total_genres_list += g.split('|')
pd.Series(total_genres_list).value_counts()#.index[0]

Drama              782
Comedy             683
Thriller           597
Action             583
Adventure          416
Crime              315
Romance            308
Family             260
Science Fiction    248
Fantasy            223
Horror             176
Mystery            168
Animation          139
Music               64
History             62
War                 58
Western             20
Documentary          8
Foreign              2
dtype: int64

**12. Какого жанра среди прибыльных фильмов больше всего?**

In [106]:
profit_movie = data.loc[(data.revenue - data.budget)> 0]
total_genres_list = []
for g in profit_movie.genres: total_genres_list += g.split('|')
pd.Series(total_genres_list).value_counts().index[0]

'Drama'

**13. Кто из режиссеров снял больше всего фильмов?**

In [122]:
data.director[data.director.str.contains(',')]

68      George Tillman, Jr.
1865    George Tillman, Jr.
Name: director, dtype: object

In [108]:
data.director.value_counts()

Steven Soderbergh            13
Ridley Scott                 12
Clint Eastwood               12
Steven Spielberg             10
Shawn Levy                   10
                             ..
David Mamet                   1
Mort Nathan                   1
Lars von Trier                1
Chris Wedge                   1
Nathan Greno|Byron Howard     1
Name: director, Length: 958, dtype: int64

**14. Кто из режиссеров снял больше всего Прибыльных фильмов?**

In [124]:
profit_movie = data.loc[(data.revenue - data.budget)> 0]
profit_movie.director.value_counts().index[0]

'Ridley Scott'

**15. Кто из режиссеров принес больше всего прибыли?**

In [134]:
data['profit'] = data.revenue - data.budget
data.groupby('director')['profit'].sum().idxmax()

'Peter Jackson'

**16. Какой актер принес больше всего прибыли?**

In [135]:
data.cast.                             

0       Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...
1       Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...
2       Shailene Woodley|Theo James|Kate Winslet|Ansel...
3       Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...
4       Vin Diesel|Paul Walker|Jason Statham|Michelle ...
                              ...                        
1885    Patrick Stewart|Hugh Jackman|Ian McKellen|Hall...
1886    Richard E. Grant|Jonathan Lipnicki|Jim Carter|...
1887    Rene Russo|Jason Alexander|Piper Perabo|Randy ...
1888    Meg Ryan|Diane Keaton|Lisa Kudrow|Walter Matth...
1889    Susan Ward|Lori Heuring|Matthew Settle|Nathan ...
Name: cast, Length: 1890, dtype: object

In [136]:
data.cast.apply(lambda x: x.split('|'))

0       [Chris Pratt, Bryce Dallas Howard, Irrfan Khan...
1       [Tom Hardy, Charlize Theron, Hugh Keays-Byrne,...
2       [Shailene Woodley, Theo James, Kate Winslet, A...
3       [Harrison Ford, Mark Hamill, Carrie Fisher, Ad...
4       [Vin Diesel, Paul Walker, Jason Statham, Miche...
                              ...                        
1885    [Patrick Stewart, Hugh Jackman, Ian McKellen, ...
1886    [Richard E. Grant, Jonathan Lipnicki, Jim Cart...
1887    [Rene Russo, Jason Alexander, Piper Perabo, Ra...
1888    [Meg Ryan, Diane Keaton, Lisa Kudrow, Walter M...
1889    [Susan Ward, Lori Heuring, Matthew Settle, Nat...
Name: cast, Length: 1890, dtype: object

In [149]:
total_acters_list = []
for cst in data.cast : total_acters_list += cst.split('|')
acters = pd.Series(total_acters_list).unique()

In [150]:
acters

array(['Chris Pratt', 'Bryce Dallas Howard', 'Irrfan Khan', ...,
       'Matthew Settle', 'Nathan Bexton', 'Tess Harper'], dtype=object)